In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.DSSM.get_default_preprocessor(ngram_size=3)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5077.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3197.89it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2082963/2082963 [00:00<00:00, 2292391.15it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 4529.63it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3440.33it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 673348.74it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3919.79it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [6]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7f1223da8c50>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1223da8748>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f129bdbe2b0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f1220e0acc0>,
 'ngram_vocab_size': 9654}

In [7]:
triletter_callback = mz.dataloader.callbacks.Ngram(
    preprocessor, mode='aggregate')

trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    batch_size=32,
    resample=True,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=32,
    callbacks=[triletter_callback]
)

In [8]:
padding_callback = mz.models.DSSM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [9]:
model = mz.models.DSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DSSM(
  (mlp_left): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9654, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (mlp_right): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9654, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)

In [10]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-32 Loss-1.498]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3369 - normalized_discounted_cumulative_gain@5(0.0): 0.4291 - mean_average_precision(0.0): 0.3889



[Iter-64 Loss-1.319]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3275 - normalized_discounted_cumulative_gain@5(0.0): 0.4237 - mean_average_precision(0.0): 0.3844



[Iter-96 Loss-1.233]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3371 - normalized_discounted_cumulative_gain@5(0.0): 0.4278 - mean_average_precision(0.0): 0.3851



[Iter-128 Loss-1.188]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.33 - normalized_discounted_cumulative_gain@5(0.0): 0.4193 - mean_average_precision(0.0): 0.3759



[Iter-160 Loss-1.147]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3363 - normalized_discounted_cumulative_gain@5(0.0): 0.413 - mean_average_precision(0.0): 0.3761



[Iter-192 Loss-1.123]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3236 - normalized_discounted_cumulative_gain@5(0.0): 0.3902 - mean_average_precision(0.0): 0.3651



[Iter-224 Loss-1.060]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3265 - normalized_discounted_cumulative_gain@5(0.0): 0.4078 - mean_average_precision(0.0): 0.3811



[Iter-256 Loss-1.033]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.342 - normalized_discounted_cumulative_gain@5(0.0): 0.4466 - mean_average_precision(0.0): 0.4036



[Iter-288 Loss-1.013]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3029 - normalized_discounted_cumulative_gain@5(0.0): 0.3988 - mean_average_precision(0.0): 0.3703

